In [58]:
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import precision_score, recall_score
import pickle


### Reading in the Data

In [42]:
data = pd.read_csv('../users_final.csv')

In [5]:
data.head()

,user_id,topic_mode,suggested,review_mode,time_taken,correct,count_attempts,hint_used,count_hints,earned_proficiency,...,total_usage,active_from_last,one_before,two_before,three_before,four_before,five_before,six_before,seven_before,target
0,8,147,81,0,1600,192,277,8,53,16,...,230,80,0,0,0,0,0,0,0,0
1,9,73,20,0,2860,110,251,9,20,5,...,150,29,0,0,0,0,0,0,0,0
2,52,996,417,128,28680,2926,3705,33,108,195,...,3222,148,0,0,0,0,0,0,0,0
3,78,24,10,0,1630,81,171,13,49,2,...,111,172,0,0,0,0,0,0,0,0
4,80,33,16,0,579,31,35,2,7,3,...,34,35,0,0,0,0,0,0,0,1


In [6]:
data.columns


Index(['user_id', 'topic_mode', 'suggested', 'review_mode', 'time_taken',
       'correct', 'count_attempts', 'hint_used', 'count_hints',
       'earned_proficiency', 'points_earned', 'total_usage',
       'active_from_last', 'one_before', 'two_before', 'three_before',
       'four_before', 'five_before', 'six_before', 'seven_before', 'target'],
      dtype='object')

In [43]:
flask_data = data[['correct', 'time_taken','count_attempts','active_from_last', 'target']]

In [45]:
flask_data['time_taken'] = (flask_data['time_taken'] / 60)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
flask_data.head()

,correct,time_taken,count_attempts,active_from_last,target
0,192,26.666667,277,80,0
1,110,47.666667,251,29,0
2,2926,478.000000,3705,148,0
3,81,27.166667,171,172,0
4,31,9.650000,35,35,1


### Setting Up the Model

In [72]:
y = flask_data['target'].values
X = flask_data.iloc[:,:-1].values
X_train, X_test, y_train, y_test = train_test_split(X,y)


gbc = GradientBoostingClassifier(learning_rate =.25, n_estimators= 100,
                    max_features= 'sqrt',
                     max_depth= 4,
                     min_samples_leaf= 1,
                     min_samples_split= 2)
gbc.fit(X_train, y_train)


GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.25, loss='deviance', max_depth=4,
                           max_features='sqrt', max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='auto',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [73]:
y_hat = gbc.predict(X_test)

In [53]:
probs = gbc.predict_proba(X)

In [54]:
probs[0][1]

0.11298627341483301

In [40]:
pickle.dump(gbc, open('gbc.sav', 'wb'))

In [39]:
gbc.predict_proba(np.array([[2926,478.000000,705,148]]))[0][1]

0.45000061464011565

In [27]:
X_test[0]

array([185277,      2,      1,      0,    331,      1,      4,      3,
            6,      0,    450,      4,     67,      0,      0,      0,
            0,      0,      0,      0])

In [74]:
gbc.score(X_test,y_test)

0.8477021133847702

In [75]:
precision_score(y_test, y_hat)

0.723091976516634

In [76]:
recall_score(y_test, y_hat)

0.5417888563049853